### Java Snowpark UDTF Examples

In this example, we will build a java udf that generates fake data. We'll also show logging and tracing.

Much of the work for this will be done outside of this notebook. It is here mostly for reference.

### Build your environment.

In IntelliJ, I'm going to create a new maven project. I'm going to create a single Java file, in this case, I'll call it FakeData and the package doesn't matter, just remember it for next time.

This class won't do a whole lot, but will use the Faker library.

The pom.xml is below you may need to update based on your project name and setup


```
<?xml version="1.0" encoding="UTF-8"?>
<?xml version="1.0" encoding="UTF-8"?>
<project xmlns="http://maven.apache.org/POM/4.0.0"
         xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
         xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 http://maven.apache.org/xsd/maven-4.0.0.xsd">
    <modelVersion>4.0.0</modelVersion>

    <groupId>com.snowflake.bharris</groupId>
    <artifactId>FebSnowcamp</artifactId>
    <version>1.0-SNAPSHOT</version>

    <properties>
        <maven.compiler.source>11</maven.compiler.source>
        <maven.compiler.target>11</maven.compiler.target>
        <project.build.sourceEncoding>UTF-8</project.build.sourceEncoding>
    </properties>

</project>
```

Now, we're going to create a very simple "Hello World" class to start, just to make sure we've got everything setup.

This is optional, you can always skip ahead if you want.

Go ahead and create a class named FakeDataUDF.java in whatever package you want.

It should look something like this:

```
package com.snowflake.bharris.snowcamp;

public class FakeDataUDF {
    public static void main(String[] args)
    {
        // Snowflake won't call this
        // It's only here for testing locally
        FakeDataUDF udf = new FakeDataUDF();
        System.out.println(udf.sayHello("Brad"));
    }

    private String sayHello(String name) {
        return "Hello: " + name;
    }

}

```

Now, we can run it, and we should get our simple output.

Something this simple, we could just code directly into Snowflake as a UDF. It would look something like this:

```
create or replace database JAVA_FUNCTIONS;
create or replace schema CODE;

CREATE WAREHOUSE IF NOT EXISTS JAVA_UDF_WH 
    WAREHOUSE_TYPE = STANDARD 
    WAREHOUSE_SIZE = XSMALL 
    INITIALLY_SUSPENDED = TRUE 
    AUTO_SUSPEND = 60;

CREATE OR REPLACE STAGE JAVA_FUNCTIONS.CODE.FILES
    DIRECTORY = (ENABLE = TRUE);
    
create or replace function say_hello(name varchar)
returns varchar
language java
called on null input
handler='TestFunc.sayHello'
target_path='@FILES/sayHello.jar'
as
'class TestFunc {
  public static String sayHello(String name) {
    return "Hello: " + name;
  }
}';

select say_hello('Brad');

```

Step 1
After cloning the project, run mvn package. If all goes well, this will create EnrichData-1.0-SNAPSHOT-jar-with-dependencies.jar in the target folder.

Step 2
Use the Snowflake CLI to upload the jar once it is compiled locally. Follow the instructions here to install the Snowflake CLI.

Step 3
3.a Start a SnowSQL session at the command line by running snowsql -a <YOUR_SNOWFLAKE_ACCOUNT> -u <YOUR_USER_NAME>

3.b Set the database, schema, and warehouse by running the following command.

``` use database <YOUR_DATABASE>;
use schema <YOUR_SCHEMA>;
use warehouse <YOUR_WAREHOUSE>;
3.c Upload the jar to Snowflake by running the following command.

put file:///<full-path>/target/EnrichData-1.0-SNAPSHOT-jar-with-dependencies.jar
@~/<YOUR_SNOWFLAKE_STAGE_NAME>/
auto_compress = false
overwrite = true;
3.d Create the UDFs in Snowflake by running the following commands.

create or replace function get_country_name(ip varchar)
returns varchar
language java
imports = ('@~/<YOUR_SNOWFLAKE_STAGE_NAME>/EnrichData-1.0-SNAPSHOT-jar-with-dependencies.jar')
handler = 'com.dash.enrichdata.EnrichData.getCountryName';
create or replace function get_city_name(ip varchar)
returns varchar
language java
imports = ('@~/<YOUR_SNOWFLAKE_STAGE_NAME>/EnrichData-1.0-SNAPSHOT-jar-with-dependencies.jar')
handler = 'com.dash.enrichdata.EnrichData.getCityName';
create or replace function get_decoded_url(ip varchar)
returns varchar
language java
imports = ('@~/<YOUR_SNOWFLAKE_STAGE_NAME>/EnrichData-1.0-SNAPSHOT-jar-with-dependencies.jar')
handler = 'com.dash.enrichdata.EnrichData.decodeURL';
create or replace function extract_product_name(ip varchar)
returns varchar
language java
imports = ('@~/<YOUR_SNOWFLAKE_STAGE_NAME>/EnrichData-1.0-SNAPSHOT-jar-with-dependencies.jar')
handler = 'com.dash.enrichdata.EnrichData.extractProductName';
Step 4
Use the UDFs in your SQL queries in Snowflake. For example:

SELECT *, 
get_country_name(CLIENTIP) as COUNTRY_NAME, 
get_city_name(CLIENTIP) as CITY_NAME, 
get_decoded_url(REQUEST) as DECODED_URL, 
extract_product_name(REQUEST) as PRODUCT_NAME 
FROM WEB_LOGS; ```